In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key="your_api_key", transport="rest")

model = genai.GenerativeModel('gemini-2.0-flash')

response = model.generate_content("你是谁？")
print(response.text)

我是一个大型语言模型，由 Google 训练。



In [56]:
import re
def cleaner(code):
    pat = re.compile(r'(/\*([^*]|(\*+[^*/]))*\*+/)|(//.*)')
    code = re.sub(pat, '', code)
    code = re.sub('\n', '', code)
    code = re.sub('\t', '', code)
    return code

In [ ]:
import pandas as pd
dataset = "reveal"
m1 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_train.pkl')
m2 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_val.pkl')
m3 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_test.pkl')

for df in [m1, m2, m3]:
    if "functionSource" in df.columns:
        df["func"] = df["functionSource"].apply(cleaner)
        
    if dataset == "draper":
        df["target"] = df["combine"] * 1

    if "label" in df.columns:
        df["target"] = df["label"]

    if dataset == "mvd":
        df["target"] = df["target"].apply(lambda x: 1 if x != 0 else 0)

m1 = m1[["func", "target"]]
m2 = m2[["func", "target"]]
m3 = m3[["func", "target"]]

In [ ]:
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import google.generativeai as genai

genai.configure(api_key="your_api_key", transport="rest")
model = genai.GenerativeModel('gemini-2.0-flash')

def detect_vulnerability(code: str) -> int:
    prompt = (
        "You are a cybersecurity expert analyzing code for vulnerabilities. "
        "Respond with '1' if vulnerable or '0' if safe.\n\n" + code
    )
    try:
        response = model.generate_content(prompt)
        return int(response.text.strip())
    except Exception as e:
        print(f"Error analyzing code with Gemini: {e}")
        return 0

def calculate_metrics(y_true, y_pred, dataset_name):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\nMetrics for {dataset_name}:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

def process_dataset_multithread(df: pd.DataFrame, name: str, max_workers: int = 10) -> pd.DataFrame:
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {
            executor.submit(detect_vulnerability, code): idx
            for idx, code in enumerate(df['func'])
        }
        for future in tqdm(as_completed(future_to_idx),
                           total=len(future_to_idx),
                           desc=f"Processing {name} set",
                           ncols=80):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
            except Exception:
                results[idx] = 0

    df['gemini_prediction'] = results
    if 'target' in df.columns:
        metrics = calculate_metrics(df['target'], df['gemini_prediction'], name)
        df['gemini_match'] = (df['gemini_prediction'] == df['target']).astype(int)

    return df

def process_datasets_in_parallel(datasets, names, max_workers: int = 10):
    results = {}
    for df, name in zip(datasets, names):
        print(f"\n--- Start {name} set ---")
        results[name] = process_dataset_multithread(df, name, max_workers=max_workers)
    return results

datasets = [m3]
names = ['test']
results = process_datasets_in_parallel(datasets, names, max_workers=8)
results['test'].to_pickle(f'../../data/finetune/{dataset}/{dataset}_test_with_gemini.pkl')
for name, df in results.items():
    calculate_metrics(df['target'], df['gemini_prediction'], name)


--- Start test set ---


Processing test set:   3%|▋                   | 72/2274 [00:07<04:35,  7.98it/s]

Error analyzing code with Gemini: invalid literal for int() with base 10: '```cpp\n1\n```'


Processing test set:   4%|▊                  | 101/2274 [00:10<03:52,  9.34it/s]

Error analyzing code with Gemini: invalid literal for int() with base 10: "```cpp\nstatic int h263_skip_b_part ( MpegEncContext * s , int cbp ) {\n  LOCAL_ALIGNED_16 ( int16_t , dblock , [ 64 ] ) ;\n  int i , mbi ;\n  mbi = s -> mb_intra ;\n  s -> mb_intra = 0 ;\n  for ( i 


...

Processing test set:  94%|████████████████▉ | 2146/2274 [03:22<00:10, 12.08it/s]

Error analyzing code with Gemini: invalid literal for int() with base 10: '```c\n1\n```'


Processing test set: 100%|██████████████████| 2274/2274 [03:33<00:00, 10.63it/s]

Error analyzing code with Gemini: invalid literal for int() with base 10: "```c\n1\n```\n\n**Explanation:**\n\nThe vulnerability lies in the potential for `avio_get_str` to read less than `size` bytes from the input, but the code doesn't account for a null terminator being 

Metrics for test:
Accuracy:  0.6003
Precision: 0.1075
Recall:    0.4043
F1 Score:  0.1699

Metrics for test:
Accuracy:  0.6003
Precision: 0.1075
Recall:    0.4043
F1 Score:  0.1699



/var/folders/zr/c6223m_9071ckmbfh1gxb27r0000gn/T/ipykernel_29721/2908444546.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gemini_prediction'] = results
/var/folders/zr/c6223m_9071ckmbfh1gxb27r0000gn/T/ipykernel_29721/2908444546.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gemini_match'] = (df['gemini_prediction'] == df['target']).astype(int)
